In [1]:
%load_ext autoreload
%autoreload 2

import src.ksd.models as models
from src.ksd.find_modes import find_modes, pairwise_directions, run_bfgs
from src.ksd.langevin import RandomWalkMH, RandomWalkBarker
import src.ksd.langevin as mcmc
from src.ksd.ksd import KSD
from src.ksd.kernel import IMQ
from src.ksd.bootstrap import Bootstrap
from src.ksd.find_modes import find_modes, pairwise_directions
from tqdm.notebook import tqdm, trange

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import matplotlib.pyplot as plt
import seaborn as sns

import src.kgof.density as density
import src.kgof.data as data

from src.nf_maf import init_model, show

MCMCKernel = RandomWalkMH # RandomWalkBarker 

In [2]:
checkpoint, maf, batched_train_data = init_model()

2022-04-13 18:55:33.372837: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-13 18:55:33.372900: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: nvidia4
2022-04-13 18:55:33.372906: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: nvidia4
2022-04-13 18:55:33.373061: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.27.4
2022-04-13 18:55:33.373092: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.27.4
2022-04-13 18:55:33.373097: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.27.4
2022-04-13 18:55:33.373833: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in pe

num in classes in train: [(0, 4932), (1, 5678)]
num in classes in val: [(0, 991), (1, 1064)]
num in classes in test: [(0, 980), (1, 1135)]


2022-04-13 18:55:33.886850: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled


Successfully initialized!
Successfully loaded trained model!


In [3]:
xx = maf.sample(10)

In [62]:
xx.shape

TensorShape([10, 28, 28])

In [64]:
show(xx)

In [76]:
xx_orig = tf.identity(xx)

In [80]:
xx = tf.convert_to_tensor(xx_orig, dtype=tf.float32)
# xx

In [85]:
with tf.GradientTape() as g:
    g.watch(xx)
    ll = tf.reduce_sum(maf.log_prob(xx))

gg = g.gradient(ll, xx)
gg[:2]

<tf.Tensor: shape=(2, 28, 28), dtype=float32, numpy=
array([[[-7.4110522e+08, -2.7321490e+09, -4.8901002e+09, ...,
          4.6334566e+09,  6.9905418e+09, -6.5231570e+09],
        [-7.3135462e+09,  9.9679171e+09,  1.2854808e+11, ...,
          3.9617446e+10, -1.7979771e+10, -1.4283122e+10],
        [ 2.2583882e+10,  2.5827489e+09, -1.3882862e+10, ...,
          5.6513237e+10,  1.5028539e+10,  3.3719091e+10],
        ...,
        [-1.0478778e+10, -1.6964180e+11,  1.4208826e+10, ...,
          5.7636069e+10, -2.4373962e+09,  3.4707820e+11],
        [ 4.9107558e+10, -1.1279250e+10, -8.6099296e+10, ...,
         -5.0653327e+10,  5.2843438e+10, -4.9387494e+11],
        [-1.4443377e+11, -2.6590026e+10, -2.2719773e+10, ...,
         -1.1086394e+10, -1.5583674e+10,  5.1940516e+10]],

       [[-2.2653535e+09, -4.8806932e+09, -9.1292600e+09, ...,
         -9.9990733e+09, -2.0031934e+10,  2.8840474e+09],
        [-9.3842790e+09, -3.8108103e+10,  1.5733010e+11, ...,
          2.5241285e+09, -4.24

# KSD test

In [3]:
# can only run once!
log_prob_old = maf.log_prob

def maf_log_prob(img):
    img = tf.reshape(img, (-1, 28, 28))
    return log_prob_old(img)

maf.log_prob = maf_log_prob

In [5]:
tf.random.set_seed(1)

n = 100

In [6]:
sample_batch = next(iter(batched_train_data))
ind = np.random.randint(0, sample_batch.shape[0], size=n).tolist()
sample_off = np.take(sample_batch, ind, axis=0)
sample_off = tf.reshape(sample_off, (-1, 28*28))

2022-04-13 18:55:39.725903: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled


In [7]:
sample_on = tf.convert_to_tensor(maf.sample(n), dtype=tf.float32)
sample_on = tf.reshape(sample_on, (-1, 28*28))

In [9]:
num_boot = 800
alpha = 0.05
nrep = 1

kernel = IMQ(med_heuristic=True)
ksd = KSD(target=maf, kernel=kernel)

In [10]:
# initialise bootstrap
bootstrap_nopert = Bootstrap(ksd, n)
multinom_samples_notrain = bootstrap_nopert.multinom.sample((nrep, num_boot)) # nrep x num_boot x n

2022-04-13 18:56:44.162608: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [15]:
## KSD
multinom_one_sample = multinom_samples_notrain[0, :] # nrep x num_boost x n

# compute p-value
rej_on, pval_on = bootstrap_nopert.test_once(
    alpha=alpha, num_boot=num_boot,X=sample_on, multinom_samples=multinom_one_sample)

rej_off, pval_off = bootstrap_nopert.test_once(
    alpha=alpha, num_boot=num_boot,X=sample_off, multinom_samples=multinom_one_sample)

In [16]:
pval_on = pva_on
pval_off = pva_off

In [17]:
print("on p-value:", pval_on, "off p-value:", pval_off)

on p-value: 0.05368289637952559 off p-value: 0.7191011235955056
